### Simple option to generate parameter files

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import glob

In [19]:
# Load file with parameter ranges
#csv='ctsm6oaat_rangetests_key.csv'
csv='example_paramfile_mods.csv'
params = pd.read_csv(csv)

In [21]:
# where do you want to save the parameter files?
out_dir = '/glade/work/linnia/CLM6-PPE/ctsm6_cal/paramfiles/'

In [22]:
# base parameter file to start with:
basefile = '/glade/work/linnia/CLM6-PPE/ctsm6_cal/paramfiles/caltest0001.nc'
base_paramfile = xr.open_dataset(basefile,decode_times=False)

In [26]:
# Modify parameter settings and write paramfiles
for ens in range(len(params)):
    new_paramfile = base_paramfile.copy()
    param = params['param_name'][ens]
    values = params['value'][ens]
    float_list = [float(v) for v in values.split(',')]
    if len(float_list)>1: # PFT indexed
        print(param,values)

        if param in ('kmax','krmax','psi50'): # write across segment
            array = base_paramfile[param].values[0,:]
            array[:len(float_list)] = float_list[0:len(float_list)]
            for seg in range(4):
                new_paramfile[param].isel(segment=seg).data = array
        else:
            array = base_paramfile[param].values
            array[:len(float_list)] = float_list[0:len(float_list)]
            new_paramfile[param].data = array
    else:
        print(param,float_list[0])
        new_paramfile[param].data = float_list[0]

    new_paramfile.to_netcdf(out_dir+params['key'][ens]+'.nc')
    

fsr_pft 0,0.075,0.075,0.075,0.1575,0.075,0.1575,0.075,0.075,0.075,0.075,0.0825,0.0825,0.1575,0.2325,0.1575,0.1575
d_max 6.1
kmax 0,0.125,0.125,0.125,0.2625,0.125,0.2625,0.125,0.125,0.125,0.125,0.1375,0.1375,0.2625,0.3875,0.2625,0.2625


### Simple parameter file modification

In [2]:
# default parameter file to start with:
basefile = '/glade/work/linnia/CLM6-PPE/ctsm6_cal/paramfiles/caltest0000.nc'
def_paramfile = xr.open_dataset(basefile,decode_times=False)

/glade/derecho/scratch/linnia/tmp/ipykernel_23877/2465774829.py:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  def_paramfile = xr.open_dataset(basefile)


In [3]:
newparams = def_paramfile.copy()

In [4]:
newparams['e_ice'] = 0.1
newparams['xdrdt'] = 0.5
newparams['upplim_destruct_metamorph'] = 250.0
newparams['lmrha'] = 23195.0
newparams['leafcn'][1:16] = newparams['leafcn'][1:16].values*1.3
newparams['jmaxha'] = 78000.0
newparams['jmaxb0'][0:] = np.repeat(0.0264,79)
newparams['jmaxb1'][0:] = np.repeat(0.09,79)

In [33]:
newparams['slatop']

<xarray.DataArray 'slatop' (pft: 79)> Size: 632B
[79 values with dtype=float64]
Coordinates:
    pftname  (pft) |S40 3kB ...
Dimensions without coordinates: pft
Attributes:
    long_name:  Specific Leaf Area (SLA) at top of canopy, projected area basis
    units:      m^2/gC

In [5]:
newparams.to_netcdf('/glade/work/linnia/CLM6-PPE/ctsm6_cal/paramfiles/caltest0056.nc')

OverflowError: Not possible to cast encoded times from dtype('int64') to dtype('int32') without overflow. Consider removing the dtype encoding, at which point xarray will make an appropriate choice, or explicitly switching to a larger integer dtype.

In [12]:
newparams['kc_nonmyc'][14] = 0.36
newparams['kn_nonmyc'][14] = 21.6
newparams['akc_active'][14] = 1.8
newparams['akn_active'][14] = 3.6
newparams['ekc_active'][14] = 10.8
newparams['ekn_active'][14] = 1.8